Get spacy and gensim:

``conda install -c conda-forge spacy``\
``python -m spacy download en_core_web_lg``\
``conda install -c conda-forge gensim``


In [1]:
import pandas as pd

df = pd.read_csv('mbti_1.csv')
# split_posts = df['posts'].to_string().split('|||')

In [2]:
import numpy as np

# break up each row into its constituent sentences
# drop any row that has more than 50 sentences (these often have ||||||| in them)
l = df['posts'].values.tolist()
targets = df['type'].values.tolist()
new_list_1 = [x.split('|||') for x in l]

for i in range(len(new_list_1)):
    new_list_1[i].insert(0, targets[i])
    
new_list = [x for x in new_list_1 if len(x) <= 51]
df_mbti = pd.DataFrame(new_list)


In [39]:
df_mbti

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw,http://41.media.tumblr.com/tumblr_lfouy03PMA1q...,enfp and intj moments https://www.youtube.com...,What has been the most life-changing experienc...,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,May the PerC Experience immerse you.,The last thing my INFJ friend posted on his fa...,Hello ENFJ7. Sorry to hear of your distress. I...,84389 84390 http://wallpaperpassion.com/uplo...,...,http://www.serebii.net/potw-dp/Scizor.jpg,Not all artists are artists because they draw....,"Welcome to the robot ranks, person who downed ...",Banned for taking all the room under my bed. Y...,http://www.youtube.com/watch?v=w8IgImn57aQ,"Banned for being too much of a thundering, gru...",Ahh... old high school music I haven't heard i...,I failed a public speaking class a few years a...,I like this person's mentality. He's a confirm...,Move to the Denver area and start a new life f...
1,ENTP,'I'm finding the lack of me in these posts ver...,Sex can be boring if it's in the same position...,Giving new meaning to 'Game' theory.,Hello *ENTP Grin* That's all it takes. Than w...,This + Lack of Balance and Hand Eye Coordination.,Real IQ test I score 127. Internet IQ tests ar...,You know you're an ENTP when you vanish from a...,http://img188.imageshack.us/img188/6422/6020d1...,http://img.adultdvdtalk.com/813a0c6243814cab84c51,...,Because its hard to be sad about losing someon...,"Oh, you don't have to tell me that most of the...",:D I'm a Nightowl. I wake up between 6-7pm and...,Personal opinion backed by theory would sugges...,Personal stocks that I have on my desktop that...,I'll tell you when I open photoshop. :) Glad ...,:D Thanks.,http://i817.photobucket.com/albums/zz96/kamioo...,:) Static: http://i817.photobucket.com/albums/...,Psychologist don't keep me around long enough ...
2,INTP,'Good one _____ https://www.youtube.com/wat...,"Of course, to which I say I know; that's my bl...",Does being absolutely positive that you and yo...,"No, I didn't; thank you for a link!",So-called Ti-Si loop (and it can stem from any...,Have you noticed how peculiar vegetation can b...,The Smiths – Never Had No One Ever,I often find myself spotting faces on marble t...,This 5 year-old sentence is an incredibly accu...,...,That happens. And it occurs because most often...,246386,"I study graphic design now, which I really enj...",Alexxxandra97 - DeviantArt,236994,http://www.youtube.com/watch?v=2Nkcn8m9M0M,I am always ready to discipline (to intimidate...,World domination? Shooting people in the head?...,218106,ISTP? http://www.youtube.com/watch?v=7ghqoYx...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...","Dear ENTJ sub, Long time no see. Sincerely,...",None of them. All other types hurt in deep exi...,Probably a sliding scale that depends on indiv...,Draco Malfoy also. I'd say he's either 358 or ...,"I'm either 358 or 385, though in which stackin...",I'm not particularly introverted or extraverte...,"Dear Type 9 INFP, Your absolute admiration of...",2% still means about 1/50 people. I've probabl...,...,"I wanted to come up with something clever, but...",So then why do you even bother asking if you'v...,Wasn't sure I listened to sx music until now. ...,INFP messaged me with a picture of herself bef...,"Either way, you see my point. The actual rarit...",WOW my math was off! I literally just said num...,Rarity in the context of the entire planet is ...,One could say that whatever the culmination of...,"I would agree, however, those that profess to ...",Lol.'
4,ENTJ,'You're fired.,That's another silly misconception. That appro...,But guys... he REALLY wants to go on a super-d...,Never mind. Just go on permanent vacation.,Two months? I wouldn't be crazy about the idea...,Lol. Its not like our views were unsolicited. ...,Sometimes I just really like impoverished rap ...,I probably would have lost it too. His stonewa...,I would have just taken the project and gave t

In [3]:
from sklearn.model_selection import train_test_split

text_X_train, text_X_test, text_y_train, text_y_test = train_test_split(df_mbti.drop(columns=[0]),
                                                    df_mbti[0],
                                                    test_size = 0.2,
                                                    random_state = 123)

# reshape train data so each sentence is on its own row, remove Nones
text_X_train = text_X_train.to_numpy().reshape(-1,1)
text_y_train = np.c_[[text_y_train for i in range(50)]].T.reshape(-1,1)
text_train_data = np.c_[text_y_train, text_X_train]
text_train_data = text_train_data[text_train_data[:,1] != None]

text_X_test = text_X_test.to_numpy().reshape(-1,1)
text_y_test = np.c_[[text_y_test for i in range(50)]].T.reshape(-1,1)
text_test_data = np.c_[text_y_test, text_X_test]
text_test_data = text_test_data[text_test_data[:,1] != None]



In [47]:
y_test = [0 for x in text_test_data[:,0]]
y_1 = [np.zeros(4) for x in text_test_data[:,0]]
for i in range(len(y_test)):
    if text_test_data[i,0][0] == "I":
        y_test[i] += 1
        y_1[i][0] += 1
    if text_test_data[i,0][1] == "S":
        y_test[i] += 2
        y_1[i][1] += 1
    if text_test_data[i,0][2] == "T":
        y_test[i] += 4
        y_1[i][2] += 1
    if text_test_data[i,0][3] == "P":
        y_test[i] += 8
        y_1[i][3] += 1
        
y_train = [0 for x in text_train_data[:,0]]
y_2 = [np.zeros(4) for x in text_train_data[:,0]]
for i in range(len(y_train)):
    if text_train_data[i,0][0] == "I":
        y_train[i] += 1
        y_2[i][0] += 1
    if text_train_data[i,0][1] == "S":
        y_train[i] += 2
        y_2[i][1] += 1
    if text_train_data[i,0][2] == "T":
        y_train[i] += 4
        y_2[i][2] += 1
    if text_train_data[i,0][3] == "P":
        y_train[i] += 8
        y_2[i][3] += 1

y_2 = np.array(y_2)
y_1 = np.array(y_1)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,0])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,0]))

0.7526058514844368


In [30]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,1])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,1]))

0.8526465866340786


In [31]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,2])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,2]))

0.5962188196391345


In [48]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,3])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,3]))

0.6134354108259651


In [50]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:

names = ["Nearest Neighbors","Decision Tree", "Random Forest", "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)


scores = []
for name, clf in zip(names, classifiers):
    print(name)
    clf.fit(X_train_tfidf, y_2[:,0])
    scores.append(clf.score(X_new_tfidf, y_1[:,0]))


Nearest Neighbors
